In [ ]:
import random
import string
import datetime
import itertools
import pandas as pd
from sqlalchemy import create_engine, text

# ==========================================
# 1. CONFIGURACIÓN DE CONEXIÓN (NECESARIA AL INICIO)
# ==========================================
# Es necesario conectar primero para saber en qué ID va la base de datos
DB_USER = "etl_user"
DB_PASSWORD = "TuPasswordFuerte"
DB_HOST = "192.168.0.103"
DB_NAME = "db_gestion"

# String de conexión
connection_string = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:3307/{DB_NAME}"
try:
    engine = create_engine(connection_string)
    print(f"Conectado a {DB_NAME} para obtener últimos IDs...")
except Exception as e:
    print("Error de conexión. Asegúrate de tener los drivers instalados (pip install mysql-connector-python sqlalchemy)")
    raise e

# ==========================================
# 2. FUNCIÓN PARA OBTENER EL ÚLTIMO ID
# ==========================================
def obtener_siguiente_id(nombre_tabla, nombre_columna_id):
    """
    Consulta la base de datos para obtener el MAX(id).
    Retorna MAX(id) + 1. Si la tabla está vacía, retorna 1.
    """
    try:
        query = text(f"SELECT MAX({nombre_columna_id}) FROM {nombre_tabla}")
        with engine.connect() as conn:
            result = conn.execute(query).scalar()
            if result is None:
                return 1
            return int(result) + 1
    except Exception as e:
        print(f"Advertencia: No se pudo leer la tabla {nombre_tabla}, asumiendo inicio en 1. Error: {e}")
        return 1

# ==========================================
# 3. OBTENCIÓN DE OFFSETS (PUNTOS DE PARTIDA)
# ==========================================
# Buscamos dónde termina la data actual para empezar justo después
start_cliente = obtener_siguiente_id('cliente', 'idCliente')
start_equipo = obtener_siguiente_id('equipo', 'idEquipo')
start_empleado = obtener_siguiente_id('empleado', 'idEmpleado')
start_estadistica = obtener_siguiente_id('estadisticas_proyecto', 'idEstadistica')
start_proyecto = obtener_siguiente_id('proyecto', 'idProyecto')
start_tarea = obtener_siguiente_id('tarea', 'idTarea')
start_asignacion = obtener_siguiente_id('asignacion_tarea', 'idAsignacion')
start_incidente = obtener_siguiente_id('incidente', 'idIncidente')

print(f"--- Puntos de inicio detectados ---")
print(f"Clientes inicia en: {start_cliente}")
print(f"Proyectos inicia en: {start_proyecto}")
print(f"Tareas inicia en: {start_tarea}")
print(f"-----------------------------------")

# ==========================================
# 4. CONFIGURACIÓN DE CANTIDADES A GENERAR
# ==========================================
N_CLIENTES = 10
N_EQUIPOS = 5
N_EMPLEADOS = 30
N_PROYECTOS = 150
N_TAREAS = 200
N_ASIGNACIONES = 100
N_ESTADISTICAS = 20
N_INCIDENTES = 120

# ==========================================
# 5. GENERADORES BÁSICOS (Lógica Original)
# ==========================================
def random_email(name):
    dominios = ['gmail.com', 'outlook.com', 'empresa.com']
    return f'{name.lower()}{random.randint(1,99)}@{random.choice(dominios)}'

def random_phone():
    return f'+52{random.randint(2220000000, 2299999999)}'

def random_industria():
    return random.choice(['Fintech', 'Educación', 'Salud', 'Retail', 'Transporte'])

def random_nombre():
    return ''.join(random.choices(string.ascii_uppercase, k=6))

def random_equipo():
    return f'Equipo_{random.choice(string.ascii_uppercase)}_{random.randint(10,99)}'

def random_estado_tarea():
    return random.choice(['PENDIENTE', 'EN_PROGRESO', 'EN_REVISION', 'COMPLETADA', 'BLOQUEADA'])

def random_estado_proyecto():
    return random.choice(['ACTIVO', 'CANCELADO', 'EN_PRUEBAS', 'EN_PLANEACION', 'FINALIZADO', 'PAUSADO'])

def random_prioridad():
    return random.choice(['BAJA', 'MEDIA', 'ALTA', 'CRITICA'])

def random_tipo_proyecto():
    return random.choice(['WEB', 'MOVIL', 'ESCRITORIO', 'EMBEBIDO'])

def random_fecha(base=datetime.date.today()):
    return base - datetime.timedelta(days=random.randint(1,1000))

# ==========================================
# 6. GENERACIÓN DE DATOS (Usando los Offsets)
# ==========================================

# --- Clientes ---
# Rango: Desde start_cliente hasta start_cliente + N
rango_clientes = range(start_cliente, start_cliente + N_CLIENTES)
clientes = []
for i in rango_clientes:
    nombre = random_nombre()
    clientes.append([
        i,
        nombre,
        random_email(nombre),
        random_phone(),
        random_industria(),
        round(random.uniform(50000, 200000), 2)
    ])

clientes_df = pd.DataFrame(clientes, columns=['idCliente','Nombre','Email','Telefono','Industria','MetricaClienteInicial'])

# --- Equipos ---
rango_equipos = range(start_equipo, start_equipo + N_EQUIPOS)
equipos = []
for i in rango_equipos:
    equipos.append([
        i,
        random_equipo(),
        random.choice([0,1])
    ])
equipos_df = pd.DataFrame(equipos, columns=['idEquipo','Nombre','Activo'])

# --- Empleados ---
rango_empleados = range(start_empleado, start_empleado + N_EMPLEADOS)
empleados = []
for i in rango_empleados:
    nombre = random_nombre()
    # FK: Asignar a uno de los equipos NUEVOS generados en este lote
    # Nota: Si quisieras asignar a equipos viejos, tendrías que consultar sus IDs, 
    # pero aquí asumimos integridad por lote.
    fk_equipo = random.choice(rango_equipos) 
    
    empleados.append([
        i,
        nombre,
        random_email(nombre),
        round(random.uniform(8000,22000), 2),
        round(random.uniform(100,300), 2),
        fk_equipo
    ])
empleados_df = pd.DataFrame(empleados, columns=['idEmpleado','Nombre','Email','Salario','SalarioxHora','Equipo_idEquipo'])

# --- Estadísticas ---
rango_estadisticas = range(start_estadistica, start_estadistica + N_ESTADISTICAS)
estadisticas = []
for i in rango_estadisticas:
    estadisticas.append([
        i,
        random_fecha(),
        random.randint(5,100),
        random.randint(0,25),
        round(random.uniform(10,100), 2),
        round(random.uniform(500, 20000), 2)
    ])
estadisticas_df = pd.DataFrame(estadisticas, columns=['idEstadistica','Fecha','Tareas_completadas','Tareas_pendientes','Horas_trabajadas','Costo_diario'])

# --- Proyectos ---
rango_proyectos = range(start_proyecto, start_proyecto + N_PROYECTOS)
proyectos = []
for i in rango_proyectos:
    fecha_inicio = random_fecha()
    duracion_estimada = random.randint(30, 180)
    fecha_fin_estimada = fecha_inicio + datetime.timedelta(days=duracion_estimada)
    desviacion = random.randint(-10, 20)
    fecha_fin_real = fecha_fin_estimada + datetime.timedelta(days=desviacion)

    # FKs: Seleccionamos IDs de los rangos que acabamos de generar
    fk_cliente = random.choice(rango_clientes)
    fk_estadistica = random.choice(rango_estadisticas)

    proyectos.append([
        i,
        f'Proyecto_{random_nombre()}',
        random.choice(['Sistema web de ventas','App móvil','ERP','Control escolar','Dashboard BI']),
        random_tipo_proyecto(),
        fecha_inicio,
        fecha_fin_estimada,
        fecha_fin_real,
        random_estado_proyecto(),
        round(random.uniform(50000,350000),2),
        round(random.uniform(50000,350000),2),
        fk_cliente,
        fk_estadistica,
        round(random.uniform(50000,350000),2),
        random.choice([0,1])
    ])

proyectos_df = pd.DataFrame(proyectos, columns=[
    'idProyecto','Nombre','Descripcion','Tipo',
    'Fecha_inicio','Fecha_fin_estimada','Fecha_fin_real','Estado',
    'Presupuesto','Costo_real','Cliente_idCliente','Estadisticas_Proyecto_idEstadistica',
    'MetricaClienteFinal','CertificacionSeguridad'
])

# --- Tareas ---
rango_tareas = range(start_tarea, start_tarea + N_TAREAS)
tareas = []
for i in rango_tareas:
    fecha_creacion = random_fecha()
    duracion_estimada = random.randint(5, 30)
    fecha_fin_estimada = fecha_creacion + datetime.timedelta(days=duracion_estimada)
    desviacion = random.randint(-3, 5)
    fecha_fin_real = fecha_fin_estimada + datetime.timedelta(days=desviacion)

    tareas.append([
        i,
        'Tarea_'+random_nombre(),
        'Implementar '+random.choice(['frontend','backend','API','deploy','test','analytics']),
        fecha_creacion,
        fecha_fin_estimada,
        fecha_fin_real,
        random_estado_tarea(),
        random_prioridad(),
        random.randint(5,120),
        random.choice([0,1]),
        random.choice([0,1])
    ])

tareas_df = pd.DataFrame(tareas, columns=[
    'idTarea','Titulo','Descripcion','Fecha_creacion',
    'Fecha_fin_estimada','Fecha_fin_real','Estado',
    'Prioridad','Horas_estimadas','EsAutomatizacion','EsReutilizado'
])

# --- Asignaciones ---
# Generamos combinaciones usando los nuevos rangos de IDs
rango_asignaciones = range(start_asignacion, start_asignacion + N_ASIGNACIONES)
tareas_empleados = list(itertools.product(rango_tareas, rango_empleados))
random.shuffle(tareas_empleados)

# Seguridad: Si pedimos más asignaciones que combinaciones posibles
limite_asignaciones = min(N_ASIGNACIONES, len(tareas_empleados))
tareas_empleados = tareas_empleados[:limite_asignaciones]

asignaciones = []
counter_asignacion = start_asignacion

for (tarea_id, empleado_id) in tareas_empleados:
    fk_proyecto = random.choice(rango_proyectos)
    
    asignaciones.append([
        counter_asignacion,
        tarea_id,
        empleado_id,
        random_fecha(),
        random.randint(5,120),
        random.randint(5,120),
        fk_proyecto
    ])
    counter_asignacion += 1
    
asignaciones_df = pd.DataFrame(asignaciones, columns=[
    'idAsignacion','Tarea_idTarea','Empleado_idEmpleado','Fecha_asignacion',
    'Horas_estimadas','Horas_reales','Proyecto_idProyecto'
])

# --- Incidentes ---
rango_incidentes = range(start_incidente, start_incidente + N_INCIDENTES)
incidentes = []
for i in rango_incidentes:
    fk_proyecto = random.choice(rango_proyectos)
    fk_tarea = random.choice(rango_tareas)

    incidentes.append([
        i,
        fk_proyecto,
        random_fecha(),
        random.choice(['BAJA','MEDIA','ALTA','CRITICA']),
        random.choice(['CERRADO']),
        fk_tarea,
        round(random.uniform(100,2500), 2)
    ])
incidentes_df = pd.DataFrame(incidentes, columns=[
    'idIncidente','Proyecto_idProyecto','Fecha_reporte','Severidad',
    'Estado','idTarea','CostoCorreccion'
])

print("Datos sintéticos generados en memoria. Iniciando carga a SQL...")

# ==========================================
# 7. INSERCIÓN A BASE DE DATOS
# ==========================================
# Usamos if_exists='append' que es crucial para añadir sin borrar
# chunksize ayuda si N es muy grande
try:
    clientes_df.to_sql('cliente', con=engine, if_exists='append', index=False)
    print(f"Insertados {len(clientes_df)} clientes.")
    
    equipos_df.to_sql('equipo', con=engine, if_exists='append', index=False)
    print(f"Insertados {len(equipos_df)} equipos.")
    
    empleados_df.to_sql('empleado', con=engine, if_exists='append', index=False)
    print(f"Insertados {len(empleados_df)} empleados.")
    
    estadisticas_df.to_sql('estadisticas_proyecto', con=engine, if_exists='append', index=False)
    print(f"Insertadas {len(estadisticas_df)} estadísticas.")
    
    proyectos_df.to_sql('proyecto', con=engine, if_exists='append', index=False)
    print(f"Insertados {len(proyectos_df)} proyectos.")
    
    tareas_df.to_sql('tarea', con=engine, if_exists='append', index=False)
    print(f"Insertadas {len(tareas_df)} tareas.")
    
    asignaciones_df.to_sql('asignacion_tarea', con=engine, if_exists='append', index=False)
    print(f"Insertadas {len(asignaciones_df)} asignaciones.")
    
    incidentes_df.to_sql('incidente', con=engine, if_exists='append', index=False)
    print(f"Insertados {len(incidentes_df)} incidentes.")

    print("\n--- PROCESO COMPLETADO EXITOSAMENTE ---")

except Exception as e:
    print(f"\nERROR DURANTE LA INSERCIÓN SQL: {e}")


Conectado a db_gestion para obtener últimos IDs...
--- Puntos de inicio detectados ---
Clientes inicia en: 31
Proyectos inicia en: 26
Tareas inicia en: 601
-----------------------------------
Datos sintéticos generados en memoria. Iniciando carga a SQL...
Insertados 10 clientes.
Insertados 5 equipos.
Insertados 30 empleados.
Insertadas 20 estadísticas.
Insertados 100 proyectos.
Insertadas 200 tareas.
Insertadas 100 asignaciones.
Insertados 120 incidentes.

--- PROCESO COMPLETADO EXITOSAMENTE ---
